# Install Libraries

In [ ]:
#Install the necessary libraries for forecasting
!pip install statsforecast mlforecast neuralforecast nixtla -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.2/71.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.8/254.8 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.5/273.5 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.6/890.6 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Importing Libraries Required for this Notebook

In [ ]:
!pip install xgboost -q

In [ ]:
from nixtla import NixtlaClient
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsforecast import StatsForecast
from statsforecast.models import AutoTBATS, AutoARIMA, AutoETS
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from mlforecast import MLForecast
from mlforecast.target_transforms import Differences
from numba import njit
from window_ops.expanding import expanding_mean
from window_ops.rolling import rolling_mean
import lightgbm as lgb
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from mlforecast.target_transforms import AutoDifferences
from neuralforecast import NeuralForecast
from neuralforecast.models import TFT, NHITS
from neuralforecast.losses.pytorch import MAE
from nixtla import NixtlaClient
from functools import reduce
#import timesfm

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


# Data Preparation

In [ ]:
# data.columns

## Dropping the columns that wont be known in advance.

In [ ]:
# data=data[["warehouse_y", "date", "orders", "holiday_y",	"shops_closed_y",	'winter_school_holidays_y',	"school_holidays_y"]]

In [ ]:
# #rename as per requirement of the forecast libraries
# data = data.rename(columns={"warehouse_y": "unique_id", "date": "ds", "orders": "y","holiday_y":"holiday", "shops_closed_y":"shops_closed",
#                             'winter_school_holidays_y':"winter_school_holidays", "school_holidays_y":"school_holidays"})

In [ ]:
# data.head()

In [ ]:
# data.info()

In [ ]:
# data['ds'] = pd.to_datetime(data['ds'])

In [ ]:
# data["unique_id"].value_counts()

In [ ]:
# data.groupby('unique_id').count()

In [ ]:
# #Splitting the Data for Train & Test
# train = data[data['ds'] <= '2024-01-15']
# test = data[data['ds'] > '2024-01-15']

In [ ]:
# train.groupby('unique_id').count()

In [ ]:
# train.head()

In [ ]:
train=pd.read_csv("Train_df_NueralForecast.csv")
test_=pd.read_csv("Future_df_NueralForecast.csv")

In [ ]:
train['ds'] = pd.to_datetime(train['ds'])
test_['ds'] = pd.to_datetime(test_['ds'])

# SFFORECAST

###Defining Statforecast Parameters

In [ ]:
models = [AutoARIMA(season_length=7)]

sf = StatsForecast(models=models, freq='D', n_jobs=1)

In [ ]:
test_

,Unnamed: 0,unique_id,ds,holiday,shops_closed,winter_school_holidays,school_holidays,year,month,day,day_of_week,week_of_year,shops_closed_lag,Shop_opened_after_close
0,1133,Brno_1,2024-01-16,0,0,0,0,2024,1,16,1,3,0.0,0
1,1134,Brno_1,2024-01-17,0,0,0,0,2024,1,17,2,3,0.0,0
2,1135,Brno_1,2024-01-18,0,0,0,0,2024,1,18,3,3,0.0,0
3,1136,Brno_1,2024-01-19,0,0,0,0,2024,1,19,4,3,0.0,0
4,1137,Brno_1,2024-01-20,0,0,0,0,2024,1,20,5,3,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,1188,Prague_3,2024-03-11,0,0,0,0,2024,3,11,0,11,0.0,0
236,1189,Prague_3,2024-03-12,0,0,0,0,2024,3,12,1,11,0.0,0
237,1190,Prague_3,2024-03-13,0,0,0,0,2024,3,13,2,11,0.0,0
238,1191,Prague_3,2024-03-14,0,0,0,0,2024,3,14,3,11,0.0,0


In [ ]:
test_.groupby('unique_id').count()

,Unnamed: 0,ds,holiday,shops_closed,winter_school_holidays,school_holidays,year,month,day,day_of_week,week_of_year,shops_closed_lag,Shop_opened_after_close
unique_id,,,,,,,,,,,,,
Brno_1,60,60,60,60,60,60,60,60,60,60,60,60,60
Prague_1,60,60,60,60,60,60,60,60,60,60,60,60,60
Prague_2,60,60,60,60,60,60,60,60,60,60,60,60,60
Prague_3,60,60,60,60,60,60,60,60,60,60,60,60,60


In [ ]:
train=train[['unique_id', 'ds', 'y', 'holiday', 'shops_closed',
       'winter_school_holidays', 'school_holidays']]
test_=test_[['unique_id', 'ds', 'holiday', 'shops_closed',
       'winter_school_holidays', 'school_holidays']]

In [ ]:
horizon = 60
sf_fcst = sf.forecast(df=train, h=horizon, X_df=test_, level=(10,20,30,40,50,60,70,80,90))

/usr/local/lib/python3.10/dist-packages/statsforecast/core.py:494: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


In [ ]:
sf_fcst.head()

,ds,AutoARIMA,AutoARIMA-lo-90,AutoARIMA-lo-80,AutoARIMA-lo-70,AutoARIMA-lo-60,AutoARIMA-lo-50,AutoARIMA-lo-40,AutoARIMA-lo-30,AutoARIMA-lo-20,AutoARIMA-lo-10,AutoARIMA-hi-10,AutoARIMA-hi-20,AutoARIMA-hi-30,AutoARIMA-hi-40,AutoARIMA-hi-50,AutoARIMA-hi-60,AutoARIMA-hi-70,AutoARIMA-hi-80,AutoARIMA-hi-90
unique_id,,,,,,,,,,,,,,,,,,,,
Brno_1,2024-01-16,8639.712458,7833.129283,8011.280898,8131.478947,8227.008546,8308.964436,8382.563358,8450.763727,8515.479209,8578.092186,8701.332730,8763.945708,8828.661189,8896.861559,8970.460481,9052.416370,9147.945969,9268.144018,9446.295633
Brno_1,2024-01-17,8995.011922,8142.013241,8330.416740,8457.531687,8558.558617,8645.230727,8723.064961,8795.189978,8863.629566,8929.845660,9060.178185,9126.394278,9194.833867,9266.958884,9344.793117,9431.465228,9532.492158,9659.607104,9848.010604
Brno_1,2024-01-18,8998.513105,8144.708346,8333.289885,8460.524955,8561.647355,8648.401369,8726.309156,8798.502331,8867.006594,8933.285261,9063.740949,9130.019617,9198.523880,9270.717055,9348.624841,9435.378856,9536.501256,9663.736325,9852.317865
Brno_1,2024-01-19,9924.615694,9067.839036,9257.076985,9384.754930,9486.229314,9573.285300,9651.464265,9723.908728,9792.651439,9859.160807,9990.070582,10056.579950,10125.322661,10197.767124,10275.946089,10363.002075,10464.476459,10592.154404,10781.392353
Brno_1,2024-01-20,9347.466227,8486.908768,8676.981789,8805.223154,8907.145326,8994.585474,9073.109430,9145.873577,9214.919637,9281.722499,9413.209955,9480.012817,9549.058877,9621.823024,9700.346980,9787.787128,9889.709300,10017.950665,10208.023686


### Defining AutoTBATS & AutoETS

In [ ]:
models = [AutoTBATS(season_length=7),AutoETS(season_length=7)]

In [ ]:
stat_fcst = StatsForecast(models=models,freq='D')

In [ ]:
train_stat_df_ETS_TBATS = train[["unique_id", "ds", "y"]]

In [ ]:
ets_tbats = stat_fcst.forecast(df=train_stat_df_ETS_TBATS, h=60, level=(5,10,20,30,40,50,60,70,80,90,95))

/usr/local/lib/python3.10/dist-packages/statsforecast/core.py:494: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


In [ ]:
ets_tbats.head()

,ds,AutoTBATS,AutoTBATS-lo-5,AutoTBATS-lo-10,AutoTBATS-lo-20,AutoTBATS-lo-30,AutoTBATS-lo-40,AutoTBATS-lo-50,AutoTBATS-lo-60,AutoTBATS-lo-70,...,AutoETS-hi-10,AutoETS-hi-20,AutoETS-hi-30,AutoETS-hi-40,AutoETS-hi-50,AutoETS-hi-60,AutoETS-hi-70,AutoETS-hi-80,AutoETS-hi-90,AutoETS-hi-95
unique_id,,,,,,,,,,,,,,,,,,,,,
Brno_1,2024-01-16,8739.128434,8707.740073,8676.227678,8612.313588,8546.253303,8476.635713,8401.507388,8317.848435,8220.333708,...,8847.584744,8918.822496,8992.452367,9070.047160,9153.784142,9247.029242,9355.717795,9492.472812,9695.164349,9870.969028
Brno_1,2024-01-17,9129.595816,9097.824308,9065.927252,9001.232985,8934.366325,8863.898937,8787.853546,8703.173397,8604.468341,...,8979.436389,9050.674141,9124.304012,9201.898805,9285.635787,9378.880887,9487.569440,9624.324457,9827.015994,10002.820674
Brno_1,2024-01-18,8946.806403,8914.804168,8882.675475,8817.511395,8750.159147,8679.180022,8602.582386,8517.287287,8417.865431,...,9045.437563,9116.675315,9190.305185,9267.899978,9351.636960,9444.882061,9553.570613,9690.325631,9893.017167,10068.821847
Brno_1,2024-01-19,9454.302304,9422.013010,9389.596123,9323.847523,9255.891126,9184.275320,9106.990605,9020.930410,8920.616742,...,9766.374470,9838.465334,9912.976964,9991.500998,10076.240778,10170.602540,10280.592700,10418.985435,10624.104317,10802.014356
Brno_1,2024-01-20,10003.243216,9970.591964,9937.811688,9871.326055,9802.607877,9730.189267,9652.038202,9565.013284,9463.575114,...,10374.101594,10449.633891,10527.702516,10609.975087,10698.760127,10797.626479,10912.867287,11057.866539,11272.777271,11459.180282


In [ ]:
# # @title ds vs AutoARIMA

# from matplotlib import pyplot as plt
# import seaborn as sns
# def _plot_series(series, series_name, series_index=0):
#   palette = list(sns.palettes.mpl_palette('Dark2'))
#   xs = series['ds']
#   ys = series['AutoARIMA']

#   # plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

# fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
# df_sorted = sf_fcst.sort_values('ds', ascending=True)
# for i, (series_name, series) in enumerate(df_sorted.groupby('unique_id')):
#   _plot_series(series, series_name, i)
#   fig.legend(title='unique_id', bbox_to_anchor=(1, 1), loc='upper left')
# sns.despine(fig=fig, ax=ax)
# plt.xlabel('ds')
# _ = plt.ylabel('AutoARIMA')

In [ ]:
# prompt: combine sf_fcst and ets_tbats and download csv

combined_df = pd.merge(sf_fcst, ets_tbats, on=['unique_id', 'ds'], how='outer')
combined_df.to_csv('combined_forecast.csv', index=False)
from google.colab import files
files.download('combined_forecast.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# MLForecast

In [ ]:
# sf_fcst.to_csv("sf_forecast_weekly_merged.csv")
# ets_tbats.to_csv("ets_tbats_forecast_weekly_merged.csv")

In [ ]:

# from google.colab import files
# files.download("sf_forecast_weekly_merged.csv")
# files.download("ets_tbats_forecast_weekly_merged.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
models = [LinearRegression(),
    lgb.LGBMRegressor(verbosity=-1),
    xgb.XGBRegressor(),
    RandomForestRegressor(random_state=0),
]
@njit
def rolling_mean_7(x):
    return rolling_mean(x, window_size=7)
@njit
def rolling_mean_28(x):
    return rolling_mean(x, window_size=28)

mlf = MLForecast(
    models=models,
    freq='D',
    lags=[7, 14, 28, 90, 180],

    lag_transforms={
        1: [expanding_mean],
       7: [rolling_mean_7, rolling_mean_28],
        14: [rolling_mean_7, rolling_mean_28],


    },
    date_features=['year', 'month', 'day', 'quarter'],
    target_transforms=[AutoDifferences(max_diffs=2)])


In [ ]:
from mlforecast.utils import PredictionIntervals

In [ ]:
train

,unique_id,ds,y,holiday,shops_closed,winter_school_holidays,school_holidays
0,Brno_1,2020-12-06,5338,0,0,0,0
1,Brno_1,2020-12-07,5621,0,0,0,0
2,Brno_1,2020-12-08,5792,0,0,0,0
3,Brno_1,2020-12-09,5894,0,0,0,0
4,Brno_1,2020-12-10,6122,0,0,0,0
...,...,...,...,...,...,...,...
4539,Prague_3,2024-01-11,5373,0,0,0,0
4540,Prague_3,2024-01-12,6373,0,0,0,0
4541,Prague_3,2024-01-13,6044,0,0,0,0
4542,Prague_3,2024-01-14,5648,0,0,0,0


In [ ]:
#For using in predict function
future_df = test_[["unique_id", "ds", 'holiday', 'shops_closed', 'winter_school_holidays', 'school_holidays']]

In [ ]:
mlf.fit(
    train,
    id_col='unique_id',
    time_col='ds',
    target_col='y',
    static_features=[],
    prediction_intervals=PredictionIntervals(n_windows=2, h=60),

)

MLForecast(models=[LinearRegression, LGBMRegressor, XGBRegressor, RandomForestRegressor], freq=D, lag_features=['lag7', 'lag14', 'lag28', 'lag90', 'lag180', 'expanding_mean_lag1', 'rolling_mean_7_lag7', 'rolling_mean_28_lag7', 'rolling_mean_7_lag14', 'rolling_mean_28_lag14'], date_features=['year', 'month', 'day', 'quarter'], num_threads=1)

In [ ]:
levels=[5,10,20,30,40,50,60,70,80,90,95]

In [ ]:
mlforecast = mlf.predict(h=60, X_df=future_df, level=levels)

In [ ]:
mlforecast.head()

,unique_id,ds,LinearRegression,LGBMRegressor,XGBRegressor,RandomForestRegressor,LinearRegression-lo-95,LinearRegression-lo-90,LinearRegression-lo-80,LinearRegression-lo-70,...,RandomForestRegressor-hi-10,RandomForestRegressor-hi-20,RandomForestRegressor-hi-30,RandomForestRegressor-hi-40,RandomForestRegressor-hi-50,RandomForestRegressor-hi-60,RandomForestRegressor-hi-70,RandomForestRegressor-hi-80,RandomForestRegressor-hi-90,RandomForestRegressor-hi-95
0,Brno_1,2024-01-16,9031.543945,8557.212891,8598.346680,8660.919922,8781.716248,8800.833374,8839.067627,8877.301880,...,8672.850928,8684.781934,8696.712939,8746.993896,8816.449829,8885.905762,8955.361694,9024.817627,9094.273560,9129.001526
1,Brno_1,2024-01-17,9158.571289,8633.008789,8500.103516,8702.179688,8646.357056,8677.625244,8740.161621,8802.697998,...,8742.712646,8783.245605,8823.778564,8841.600537,8848.067017,8854.533496,8860.999976,8867.466455,8873.932935,8877.166174
2,Brno_1,2024-01-18,9207.705078,8777.355469,8716.870117,8789.529297,8732.261072,8758.908374,8812.202979,8865.497583,...,8829.693408,8869.857520,8910.021631,9007.409619,9133.409546,9259.409473,9385.409399,9511.409326,9637.409253,9700.409216
3,Brno_1,2024-01-19,9907.127930,9686.538086,9968.826172,9843.119141,9620.684656,9637.742358,9671.857764,9705.973169,...,9930.773193,10018.427246,10106.081299,10197.366309,10290.466797,10383.567285,10476.667773,10569.768262,10662.868750,10709.418994
4,Brno_1,2024-01-20,9465.473633,9141.835938,9249.475586,9331.088867,8858.666797,8890.338477,8953.681836,9017.025195,...,9386.492773,9441.896680,9497.300586,9569.171582,9649.276123,9729.380664,9809.485205,9889.589746,9969.694287,10009.746558


In [ ]:
# prompt: download mlforecast as csv

from google.colab import files
mlforecast.to_csv('mlforecast.csv', index=False)
files.download('mlforecast.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Neural Forecast

In [ ]:
hist_exog_list = ['holiday', 'shops_closed', 'winter_school_holidays', 'school_holidays']
levels = [5,10,20,30,40,50,60,70,80,90,95]
horizon = 60  # Number of future time steps to predict
from neuralforecast.losses.pytorch import MQLoss
# Initialize the TFT model
tft_model = TFT(h=60,
                input_size=120,
                hist_exog_list=hist_exog_list,loss=MQLoss(level=levels),
                max_steps=200,
                start_padding_enabled=True)

# Initialize the NHITS model
nhits_model = NHITS(h=60,
                input_size=120,
                hist_exog_list=hist_exog_list,loss=MQLoss(level=levels),
                max_steps=200,
                start_padding_enabled=True)

INFO:lightning_fabric.utilities.seed:Seed set to 1
INFO:lightning_fabric.utilities.seed:Seed set to 1


In [ ]:
nf = NeuralForecast(models=[tft_model, nhits_model], freq='D')

nf.fit(df=train)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name                    | Type                     | Params | Mode 
-----------------------------------------------------------------------------
0 | loss                    | MQLoss                   | 23     | train
1 | padder_train            | ConstantPad1d            | 0      | train
2 | scaler                  | TemporalNorm             | 0      | train
3 | embedding               | TFTEmbedding             | 1.5 K  | train
4 | temporal_encoder        | TemporalCovariateEncoder | 948 K  | train
5 | temporal_fusion_decoder | TemporalFusionDecoder    | 256 K  | train
6 | output_adapter          | Linear      

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=200` reached.


In [ ]:
neural_fcst = nf.predict(futr_df = future_df)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/neuralforecast/core.py:210: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


In [ ]:
neural_fcst.head()

,ds,TFT-median,TFT-lo-95,TFT-lo-90,TFT-lo-80,TFT-lo-70,TFT-lo-60,TFT-lo-50,TFT-lo-40,TFT-lo-30,...,NHITS-hi-10,NHITS-hi-20,NHITS-hi-30,NHITS-hi-40,NHITS-hi-50,NHITS-hi-60,NHITS-hi-70,NHITS-hi-80,NHITS-hi-90,NHITS-hi-95
unique_id,,,,,,,,,,,,,,,,,,,,,
Brno_1,2024-01-16,9076.627930,6892.320312,7567.018066,7837.943359,8135.625000,8283.696289,8454.389648,8560.760742,8648.168945,...,9191.197266,9331.517578,9362.164062,9394.938477,9532.582031,9722.514648,9872.685547,10198.695312,10846.224609,11385.200195
Brno_1,2024-01-17,9152.693359,6974.111328,7629.871094,7944.759766,8167.326172,8362.164062,8509.053711,8654.398438,8745.848633,...,9756.201172,9857.935547,10054.204102,10216.833984,10424.081055,10751.800781,10798.501953,11086.889648,11436.608398,11694.414062
Brno_1,2024-01-18,9178.630859,7006.703613,7658.802734,7997.097168,8175.016602,8401.835938,8531.181641,8705.817383,8794.854492,...,9659.758789,9712.970703,9891.680664,10059.553711,10281.027344,10713.671875,10835.478516,11260.566406,11503.311523,11786.779297
Brno_1,2024-01-19,9190.117188,7018.850586,7668.493652,8018.183105,8175.300293,8409.901367,8545.748047,8720.608398,8819.722656,...,9608.143555,9617.703125,9822.595703,9816.723633,9924.803711,10283.192383,10554.437500,11059.040039,11416.155273,11921.619141
Brno_1,2024-01-20,9196.773438,7021.318359,7668.572754,8025.329102,8173.090820,8408.694336,8554.038086,8722.419922,8833.048828,...,9783.053711,9875.394531,10046.551758,10121.969727,10358.913086,10552.636719,10729.541016,11071.921875,11348.045898,11741.839844


In [ ]:

neural_fcst.to_csv("neural_forecast_weekly_merged.csv")

In [ ]:

from google.colab import files

files.download("neural_forecast_weekly_merged.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# TimeGPT

In [ ]:
train.tail()

,Unnamed: 0,unique_id,ds,y,holiday,shops_closed,winter_school_holidays,school_holidays,year,month,day,day_of_week,week_of_year,shops_closed_lag,Shop_opened_after_close
4523,1128,Prague_3,2024-01-11,5373.0,0,0,0,0,2024,1,11,3,2,0.0,0
4524,1129,Prague_3,2024-01-12,6373.0,0,0,0,0,2024,1,12,4,2,0.0,0
4525,1130,Prague_3,2024-01-13,6044.0,0,0,0,0,2024,1,13,5,2,0.0,0
4526,1131,Prague_3,2024-01-14,5648.0,0,0,0,0,2024,1,14,6,2,0.0,0
4527,1132,Prague_3,2024-01-15,5387.0,0,0,0,0,2024,1,15,0,3,0.0,0


In [ ]:
train["unique_id"].value_counts()

,count
unique_id,
Brno_1,1132
Prague_1,1132
Prague_2,1132
Prague_3,1132


In [ ]:

# Initialize the NixtlaClient
nixtla_client = NixtlaClient(api_key='nixak-dZyIsJpKaqa6jjcPkhMCcHNI6S9heGWdO5iG2WixOCViM5afbymNqIRgE1XZHgkTx7MhzC9lhzFz4eKm')

# Assuming df is your input dataframe with historical data
# and future_df is your dataframe with future exogenous variables

# Specify the exogenous variables
exog_vars = ['holiday', 'shops_closed', 'winter_school_holidays', 'school_holidays']
# Make the forecast
timegpt_fcst_zero = nixtla_client.forecast(df=train, h=60, X_df=future_df, freq='D', level=(10,20,30,40,50,60,70,80,90))


In [ ]:
timegpt_fcst_zero['ds'] = pd.to_datetime(timegpt_fcst_zero['ds'])

In [ ]:
timegpt_fcst_zero.head()

,unique_id,ds,TimeGPT,TimeGPT-hi-10,TimeGPT-hi-20,TimeGPT-hi-30,TimeGPT-hi-40,TimeGPT-hi-50,TimeGPT-hi-60,TimeGPT-hi-70,...,TimeGPT-hi-90,TimeGPT-lo-10,TimeGPT-lo-20,TimeGPT-lo-30,TimeGPT-lo-40,TimeGPT-lo-50,TimeGPT-lo-60,TimeGPT-lo-70,TimeGPT-lo-80,TimeGPT-lo-90
0,Brno_1,2024-01-16,8695.377,8747.879,8792.213,8815.374,8855.829,8910.311,8949.853,9024.090,...,9433.387,8642.875,8598.541,8575.380,8534.925,8480.443,8440.901,8366.664,8249.651,7957.367
1,Brno_1,2024-01-17,9306.747,9347.514,9384.484,9415.934,9449.814,9490.608,9530.872,9642.679,...,9994.461,9265.980,9229.010,9197.561,9163.680,9122.886,9082.622,8970.815,8884.920,8619.033
2,Brno_1,2024-01-18,9424.250,9462.207,9505.508,9562.170,9601.433,9656.386,9736.619,9796.537,...,10218.851,9386.293,9342.992,9286.330,9247.067,9192.114,9111.881,9051.963,8985.793,8629.649
3,Brno_1,2024-01-19,11190.070,11232.303,11308.527,11375.873,11450.777,11536.590,11613.507,11691.202,...,12080.773,11147.838,11071.613,11004.268,10929.363,10843.551,10766.634,10688.938,10576.265,10299.367
4,Brno_1,2024-01-20,10464.600,10506.076,10540.424,10584.891,10642.564,10709.744,10738.527,10795.803,...,11244.379,10423.123,10388.775,10344.309,10286.635,10219.455,10190.672,10133.396,10021.726,9684.820


## Timesfm

In [ ]:
# Y_test_df = test.copy()
# df = train.copy()

In [ ]:
# tfm = timesfm.TimesFm(
#     context_len=512,
#     horizon_len=8,
#     input_patch_len=32,
#     output_patch_len=128,
#     num_layers=20,
#     model_dims=1280,
#     backend="cpu",
# )
# tfm.load_from_checkpoint(repo_id="google/timesfm-1.0-200m")

In [ ]:
# timesfm_df = tfm.forecast_on_df(
#     inputs=df,
#     freq='MS',
#     value_name='y',
#     num_jobs=-1,
# )

In [ ]:
timesfm_df = pd.read_csv("timesfm_weekly.csv")

In [ ]:
timesfm_df['ds'] = pd.to_datetime(timesfm_df['ds'])

In [ ]:
timegpt_fcst_zero.head()

,unique_id,ds,TimeGPT,TimeGPT-hi-10,TimeGPT-hi-20,TimeGPT-hi-30,TimeGPT-hi-40,TimeGPT-hi-5,TimeGPT-hi-50,TimeGPT-hi-60,...,TimeGPT-lo-20,TimeGPT-lo-30,TimeGPT-lo-40,TimeGPT-lo-5,TimeGPT-lo-50,TimeGPT-lo-60,TimeGPT-lo-70,TimeGPT-lo-80,TimeGPT-lo-90,TimeGPT-lo-95
0,Brno_1,2023-05-21,50947.640,51064.348,51179.760,51598.484,51630.613,51041.710,52159.880,52785.008,...,50715.520,50296.797,50264.668,50853.570,49735.402,49110.273,48768.773,48609.207,48352.297,48343.777
1,Brno_1,2023-05-28,50373.836,50975.100,51178.605,51385.230,51462.010,50814.895,52193.970,52564.120,...,49569.066,49362.440,49285.660,49932.777,48553.703,48183.550,47873.960,47588.652,47462.938,47187.260
2,Brno_1,2023-06-04,50199.540,51030.780,51623.450,52321.617,52401.227,50714.188,52419.246,53587.504,...,48775.630,48077.460,47997.850,49684.890,47979.832,46811.574,46031.027,45937.625,45610.246,45270.850
3,Brno_1,2023-06-11,50015.566,50286.242,50609.113,50628.266,50703.490,50099.547,50977.492,51579.324,...,49422.020,49402.867,49327.645,49931.586,49053.640,48451.810,47045.047,45713.050,45564.188,45347.242
4,Brno_1,2023-06-18,49981.720,50468.690,51029.082,51481.043,52208.836,50185.973,52793.062,53044.773,...,48934.355,48482.395,47754.600,49777.465,47170.375,46918.664,46099.168,45073.620,44845.730,44818.910


In [ ]:
# Make the forecast with timegpt finetuned
timegpt_fcst_finetune_4 = nixtla_client.forecast(df=train, h=60, X_df=future_df, freq='D', finetune_steps=4, level=(5,10,20,30,40,50,60,70,80,90,95))

In [ ]:
timegpt_fcst_finetune_6 = nixtla_client.forecast(df=train, h=60, X_df=future_df, freq='D', finetune_steps=6, level=(5,10,20,30,40,50,60,70,80,90,95))

In [ ]:
timegpt_fcst_finetune_8= nixtla_client.forecast(df=train, h=60, X_df=future_df, freq='D', finetune_steps=8, level=(5,10,20,30,40,50,60,70,80,90,95))

In [ ]:
timegpt_fcst_finetune_6.rename(columns={'TimeGPT': 'TimeGPT_6'}, inplace=True)

In [ ]:
timegpt_fcst_finetune_4.rename(columns={'TimeGPT': 'TimeGPT_4'}, inplace=True)

In [ ]:
timegpt_fcst_finetune_8.rename(columns={'TimeGPT': 'TimeGPT_8'}, inplace=True)

In [ ]:
timegpt_fcst_finetune_4.columns = ['TimeGPT_4-' + str(col) if 'TimeGPT-' in str(col) else col for col in timegpt_fcst_finetune_4.columns]
timegpt_fcst_finetune_6.columns = ['TimeGPT_6-' + str(col) if 'TimeGPT-' in str(col) else col for col in timegpt_fcst_finetune_6.columns]






In [ ]:


timegpt_fcst_finetune_8.columns = ['TimeGPT_8-' + str(col) if 'TimeGPT-' in str(col) else col for col in timegpt_fcst_finetune_8.columns]

In [ ]:
timesfm_df = pd.read_csv("timesfm_weekly.csv")

In [ ]:

timesfm_df['ds'] = pd.to_datetime(timesfm_df['ds'])

In [ ]:
#Merge All Results
data_frames = [sf_fcst, ets_tbats,mlforecast, neural_fcst,timegpt_fcst_zero,timegpt_fcst_finetune_4, timegpt_fcst_finetune_6, timegpt_fcst_finetune_8 ]
final_forecast_monthly_merged = reduce(lambda  left,right: pd.merge(left,right,on=['unique_id', 'ds'],how='inner'), data_frames)

In [ ]:
final_forecast_monthly_merged.to_csv("final_forecast_daily_merged.csv")

In [ ]:
# prompt: write code to download final_forecast_weekly_merged.csv to local

from google.colab import files
files.download('final_forecast_daily_merged.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>